# Studying Convergence to Absorbing States Via EigenDecomposition

## Method

Consider the Markov chain corresponding to the best-reply process with inertia. Suppose there are $n_s$ joint policies (states in the Markov chain), $n_e$ of them are equilibria or absorbing states. Suppose further that we know the transition matrix of this Markov chain (determined by the best-reply graph and the inertia parameters $\lambda^i$). Enumerate the joint policy space by the indices $0, 1, ..., n_s -1$ so that $(P)_{i,j}$ is the probability of transitioning from state $i$ to state $j$

We wish to compute the probability of terminating at each absorbing state/policy given the initial state.

These probabilities are given by the matrix:

$$P_\infty := \lim_{t\to\infty}{P^t}$$

Then, we have that $(P_\infty)_{i,j}$ gives the probability of terminating at state $j$ given that the initial state is state $i$. There is some structure that we know about $(P_\infty)_{i,j}$ immediately from the BR process. First, $(P_\infty)_{i,j} > 0$ only if $j$ is one of the $n_e$ equilibria. Also, $(P_\infty)_{i,i} = 1$ if $i$ is an equilibrium. Finally, of course, $\sum_j{(P_\infty)_{i,j}} = 1$ for all $i$. While $P_\infty$ is an $n_s \times n_s$ matrix, there are only $(n_s - n_e)\times n_e$ unknown/relevant entries. 

Let's assume that the transition matrix $P$ is diagonalizable. 

\[Q: Is this guaranteed if the inertias of each agent are different? (For the simple example studied here, $P$ is not diagonalizable when $\lambda^0 = \lambda^1 = 0.5$, but is when $\lambda^0 = 0.25, \lambda^1 = 0.75$)\]

Then:

$$P = V \Lambda V^{-1}$$

Where $\Lambda$ is the diagonal matrix of eigenvalues: $\lambda_0, ..., \lambda_{n_s}$. We know that $|\lambda_i| \le 1$ for all $i$ (confirm and justify). All eigenvalues $|\lambda| < 1$ will vanish in the limit, and the eigenvectors with an eigenvalue $\lambda = 1$ will correspond to the equilibria or absorbing states.

Studying $(P_\infty)_{i,j}$ in terms of its eigendecomposition, we have that:

$$P_\infty := \lim_{t\to\infty}{P^t} = V (\lim_{t\to\infty}{\Lambda^t}) V^{-1}$$

But, $\lim_{t\to\infty}{\Lambda^t}$ is simply the diagonal matrix of indicator functions for the condition $\lambda_i = 1$.

$$
\Lambda_\infty := \lim_{t\to\infty}{\Lambda^t} = 
\begin{bmatrix}
1_{\{\lambda_0=1\}} &  &  \\ 
 &  \ddots& \\ 
 &  & 1_{\{\lambda_{n_s}=1\}}
\end{bmatrix}
$$

Since $\Lambda_\infty$ is a diagonal matrix $0$ or $1$, $\Lambda_\infty = \Lambda_\infty^2$. So, 

$$P_\infty = (V \Lambda_\infty) (\Lambda_\infty V^{-1})$$

Let $A:= V \Lambda_\infty$ and $B:= \Lambda_\infty V^{-1}$, such that $P\infty = AB$.

Thus, $A$ is the matrix of eigenvectors corresponding to absorbing states with the rest of the eigenvectors (columns) zeroed out. Similarly, $B$ is the inverse eigenvector matrix with the rows corresponding to non-absorbing states zeroed out.

So, only the eigenvectors corresponding to $\lambda = 1$ are relevant to $A$.

\[note: more thought needed to interpret $B$ and the possible effect of non-absorbing eigenvectors\].


Then, to compute $(P_\infty)_{i,j}$, we have the equation:

$$(P_\infty)_{i,j} = \sum_{k=0}^{n_s-1}{(A)_{i,k} (B)_{k,j}}$$

But, as noted above, only columns corresponding to the absorbing states are non-zero in $A$, and similarly only rows corresponding to absorbing states are non-zero in $B$. Thus, let the set of indices corresponding to absorbing states be $E:= \{i : \lambda_i = 1\}$. Then $(P_\infty)_{i,j}$ reduces to:

$$(P_\infty)_{i,j} = \sum_{k \in E}{(A)_{i,k} (B)_{k,j}}$$

\[Q: is $|E| = n_e$ necessarily?]


## Applying to simple problem

Let us use this to compute $P_\infty$ for the simple problem we studied earlier with the following best-reply graph.

![brgraph](https://i.imgur.com/iDXX9n0.png)

In [1]:
import numpy as np
from sympy import *

from sympy.interactive import printing
from IPython.display import Math, display

This problem has the following transition matrix for its best-reply process with inertia. 

Note: states are enumerated according to the binary representation of the joint policy (i.e.: joint policy ((1,), (0,)) is state 2). State 0 and state 3 are the two equilibria (3 being the globally optimal one).

In [2]:
# define the transition matrix
P_ = np.array([[1.    , 0.    , 0.    , 0.    ],
               [0.0625, 0.1875, 0.1875, 0.5625],
               [0.5625, 0.1875, 0.1875, 0.0625],
               [0.    , 0.    , 0.    , 1.    ]])
P = Matrix(P_).applyfunc(nsimplify)
display(Math(f'P = {printing.default_latex(P)}'))

<IPython.core.display.Math object>

This matrix is diagonalizable, and admits the following eigendecomposition,

In [3]:
eigenvalues, V_ = np.linalg.eig(P_)

with eigenvector matrix $V$

In [4]:
V = Matrix(V_).applyfunc(nsimplify)
display(Math(f'V = {printing.default_latex(V)}'))

<IPython.core.display.Math object>

and diagonal eigenvalue matrix $\Lambda$

In [5]:
L = Matrix(np.diag(eigenvalues))
display(Math(f'\Lambda = {printing.default_latex(L)}'))

<IPython.core.display.Math object>

(One of the eigenvalues appears to be 0)

Thus, $\Lambda_\infty$ is the indicator for the eigenvectors corresponding to equilibrium states.

In [6]:
L_inf = Matrix(np.diag(eigenvalues >= 1).astype(int))
display(Math(f'\Lambda_\infty = {printing.default_latex(L_inf)}'))

<IPython.core.display.Math object>

Thus, in calculating $A$, only the eigenvectors corresponding to the two equilibria are relevant. This gives $A$

In [7]:
A = V * L_inf
display(Math(f'A = {printing.default_latex(A)}'))

<IPython.core.display.Math object>

similarly, B is

In [8]:
B = L_inf * V.inv()
display(Math(f'B = {printing.default_latex(B)}'))

<IPython.core.display.Math object>

Finally, now we can easily compute $P_\infty$ from $P_\infty = AB$

In [9]:
P_inf = A * B
display(Math(f'P_\infty = {printing.default_latex(P_inf)}'))

<IPython.core.display.Math object>

This exactly gives us the probability of terminating at each equilibrium given an initial state. For example, this tells us that if we begin the process at state 1, corresponding to joint policy ((0,), (1,)), we will terminate at the suboptimal equilibrium with probability $1/4$ and the globally optimal equilibrium with probability $3/4$. This has been confirmed with simulation for all initial states.

Furthermore, we can compute the probability of ending at each equilibrium given any initial distribution over states. For example, suppose the initial state is uniformly distributed. This corresponds to the row vector $\begin{bmatrix} 1/4 & 1/4 & 1/4 & 1/4 \end{bmatrix}$. Then, we have:

$$\begin{bmatrix} 1/4 & 1/4 & 1/4 & 1/4 \end{bmatrix} P_\infty = \begin{bmatrix} 1/2 & 0 & 0 & 1/2 \end{bmatrix}$$

So, there is an equal probability of terminating at either equilibrium.